In [2]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

# Load the first model: venkatd/BioMed_NER
model1_name = "venkatd/BioMed_NER"
tokenizer1 = AutoTokenizer.from_pretrained(model1_name)
model1 = AutoModelForTokenClassification.from_pretrained(model1_name)

# Load the second model: d4data/biomedical-ner-all
model2_name = "d4data/biomedical-ner-all"
tokenizer2 = AutoTokenizer.from_pretrained(model2_name)
model2 = AutoModelForTokenClassification.from_pretrained(model2_name)


In [2]:
%pip install spacy

  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached murmurhash-1.0.11-cp312-cp312-win_amd64.whl.metadata (2.0 kB)
  Using cached cymem-2.0.10-cp312-cp312-win_amd64.whl.metadata (8.6 kB)
  Using cached preshed-3.0.9-cp312-cp312-win_amd64.whl.metadata (2.2 kB)
  Using cached wasabi-1.1.3-py3-none-any.whl.metadata (28 kB)
  Using cached srsly-2.4.8-cp312-cp312-win_amd64.whl.metadata (20 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metadata (14 kB)
  Using cached weasel-0.4.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached typer-0.15.1-py3-none-any.whl.metadata (15 kB)
  Using cached langcodes-3.5.0-py3-none-any.whl.metadata (29 kB)
  Using cached language_data-1.3.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached confection-0.1.5-py3-none-any.whl.metadata (19 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached cloudpathlib-0.20.0-py3-

In [2]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch
import torch.nn.functional as F
def extract_entities(text, model, tokenizer):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    
    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs).logits
    
    # Get the predicted token labels
    predictions = torch.argmax(outputs, dim=-1)

    # Convert token IDs back to tokens
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

    # Get model labels (you can adjust based on the model’s configuration)
    labels = model.config.id2label
    
    # Extract named entities and their scores
    entities = []
    for token, prediction, logits in zip(tokens, predictions[0], outputs[0]):
        if prediction != 0:  # Assuming '0' is the 'O' tag for non-entities
            # Get the score (probability) for the predicted class
            class_score = F.softmax(logits, dim=-1)[prediction.item()].item()
            entities.append((token, labels[prediction.item()], class_score))
    
    return entities

In [11]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import pandas as pd
import tqdm
import os

# Load the first model: venkatd/BioMed_NER
model1_name = "venkatd/BioMed_NER"
tokenizer1 = AutoTokenizer.from_pretrained(model1_name)
model1 = AutoModelForTokenClassification.from_pretrained(model1_name)

# Sample biomedical text in Chinese
df = pd.read_parquet("C:\\Users\\Gaming\\Documents\\GitHub\\MIE2\\2024-fall-assignment-linaaron88\\project\\nejm\\nejm_test.parquet")
texts = df.english.tolist()
# Extract entities
english_entities = []
for text in tqdm.tqdm(texts):
    # print(f"Text: {text}")
    entities = extract_entities(text, model1, tokenizer1)
    for i in entities:
        if i[2] >= .20:
            english_entities.extend(entities)

df_entities_sentence_zh = pd.DataFrame(english_entities, columns=["word", "entity_group", "score"])
df_entities_sentence_zh.to_parquet("C:\\Users\\Gaming\\Documents\\GitHub\\MIE2\\2024-fall-assignment-linaaron88\\project\\nejm\\en-test-entities.parquet")

100%|██████████| 2102/2102 [03:51<00:00,  9.07it/s]


In [14]:
df_entities_sentence_zh

,word,entity_group,score
0,[CLS],I-Dosage,0.040192
1,ci,B-Medication,0.393350
2,mini,B-Medication,0.410082
3,b,B-Medication,0.299261
4,[SEP],I-Dosage,0.040756
...,...,...,...
193645,▁gene,B-Diagnostic_procedure,0.820578
193646,▁sequencing,I-Diagnostic_procedure,0.762487
193647,▁panel,I-Diagnostic_procedure,0.663011
193648,▁negative,B-Lab_value,0.943261


In [16]:
df = pd.read_parquet("C:\\Users\\Gaming\\Documents\\GitHub\\MIE2\\2024-fall-assignment-linaaron88\\project\\nejm\\en-entities.parquet")
df

,entity_group,score,word,start,end
0,Diagnostic_procedure,0.665001,body mass,73,83
1,Sign_symptom,0.841736,loss of muscle mass,15,35
2,Lab_value,0.456666,58,38,41
3,Diagnostic_procedure,0.208239,blood pressure,12,27
4,Clinical_event,0.731938,discharge,39,49
...,...,...,...,...,...
237137,Lab_value,0.765173,6 %,185,189
237138,Detailed_description,0.278184,AC,25,28
237139,Detailed_description,0.173570,CORD,28,32
237140,Detailed_description,0.522833,multiple combinations of,108,133


In [17]:
grouped = df[(df.word != '') & (df.word != '-') & df.entity_group != "Lab_value"][["word", "score"]].groupby(by=['word']).count().reset_index()

In [20]:
demp = grouped.sort_values("score", ascending=False)
demp

,word,score
284,.,7421
368,0,1877
639,1,1511
37914,s,1423
35053,placebo,1134
...,...,...
20277,comprehensive,1
20276,compound muscle action potential,1
20275,compound heterozygous for the,1
20274,compound heterozygous,1


In [22]:
len(demp[demp.score < 100])

44731

In [26]:
demp[demp.score > 100]

,word,score
284,.,7421
368,0,1877
639,1,1511
37914,s,1423
35053,placebo,1134
...,...,...
23890,failure,102
12013,O,101
14082,United,101
36688,rate,101


In [23]:
44731/45069

0.9925003882935055

In [24]:
dirty = pd.read_parquet("C:\\Users\\Gaming\\Documents\\GitHub\\MIE2\\2024-fall-assignment-linaaron88\\project\\nejm\\en-entities-filtered.parquet")
dirty

,word,score
0,,0.991196
1,$,0.604560
2,$ 7000,0.216629
3,%,0.978267
4,% -40 %,0.929531
...,...,...
45064,≥ 85th percentile,0.848104
45065,≥ 93 %,0.887354
45066,≥ 94 %,0.653164
45067,≥ 95th percentile,0.942542


In [10]:
english_entities

[[('[CLS]', 'I-Dosage', 0.0401923805475235),
  ('ci', 'B-Medication', 0.3933495879173279),
  ('mini', 'B-Medication', 0.4100815951824188),
  ('b', 'B-Medication', 0.29926082491874695),
  ('[SEP]', 'I-Dosage', 0.04075567051768303)],
 [('[CLS]', 'I-Dosage', 0.0401923805475235),
  ('ci', 'B-Medication', 0.3933495879173279),
  ('mini', 'B-Medication', 0.4100815951824188),
  ('b', 'B-Medication', 0.29926082491874695),
  ('[SEP]', 'I-Dosage', 0.04075567051768303)],
 [('[CLS]', 'I-Dosage', 0.0401923805475235),
  ('ci', 'B-Medication', 0.3933495879173279),
  ('mini', 'B-Medication', 0.4100815951824188),
  ('b', 'B-Medication', 0.29926082491874695),
  ('[SEP]', 'I-Dosage', 0.04075567051768303)],
 [('[CLS]', 'I-Dosage', 0.03968020901083946),
  ('▁as', 'B-Biological_structure', 0.14295202493667603),
  ('ci', 'B-Biological_structure', 0.12054912745952606),
  ('mini', 'B-Medication', 0.145082488656044),
  ('b', 'B-Medication', 0.1564876288175583),
  ('▁native', 'B-Detailed_description', 0.406858921

In [8]:
df_entities_sentence_zh

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,"([CLS], I-Dosage, 0.0401923805475235)","(ci, B-Medication, 0.3933495879173279)","(mini, B-Medication, 0.4100815951824188)","(b, B-Medication, 0.29926082491874695)","([SEP], I-Dosage, 0.04075567051768303)",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,"([CLS], I-Dosage, 0.0401923805475235)","(ci, B-Medication, 0.3933495879173279)","(mini, B-Medication, 0.4100815951824188)","(b, B-Medication, 0.29926082491874695)","([SEP], I-Dosage, 0.04075567051768303)",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,"([CLS], I-Dosage, 0.0401923805475235)","(ci, B-Medication, 0.3933495879173279)","(mini, B-Medication, 0.4100815951824188)","(b, B-Medication, 0.29926082491874695)","([SEP], I-Dosage, 0.04075567051768303)",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,"([CLS], I-Dosage, 0.03968020901083946)","(▁as, B-Biological_structure, 0.14295202493667...","(ci, B-Biological_structure, 0.12054912745952606)","(mini, B-Medication, 0.145082488656044)","(b, B-Medication, 0.1564876288175583)","(▁native, B-Detailed_description, 0.4068589210...","(▁mutated, B-Detailed_description, 0.424982577...","(▁BCR, B-Biological_structure, 0.3654800951480...","(-, B-Detailed_description, 0.29957303404808044)","(@, I-Detailed_description, 0.3370903730392456)","(▁A, I-Biological_structure, 0.22102558612823486)","(BL, B-Biological_structure, 0.3271843194961548)","(1, I-Biological_structure, 0.2973073422908783)","(▁gatekeeper, B-Detailed_description, 0.521626...","(▁T, B-Detailed_description, 0.43164002895355225)","(I, I-Detailed_description, 0.34801772236824036)","(▁mutant, I-Detailed_description, 0.4297404289...","([SEP], I-Dosage, 0.03923286125063896)",None,None
4,"([CLS], I-Dosage, 0.03968020901083946)","(▁as, B-Biological_structure, 0.14295202493667...","(ci, B-Biological_structure, 0.12054912745952606)","(mini, B-Medication, 0.145082488656044)","(b, B-Medication, 0.1564876288175583)","(▁native, B-Detailed_description, 0.4068589210...","(▁mutated, B-Detailed_description, 0.424982577...","(▁BCR, B-Biological_structure, 0.3654800951480...","(-, B-Detailed_description, 0.29957303404808044)","(@, I-Detailed_description, 0.3370903730392456)","(▁A, I-Biological_structure, 0.22102558612823486)","(BL, B-Biological_structure, 0.3271843194961548)","(1, I-Biological_structure, 0.2973073422908783)","(▁gatekeeper, B-Detailed_description, 0.521626...","(▁T, B-Detailed_description, 0.43164002895355225)","(I, I-Detailed_description, 0.34801772236824036)","(▁mutant, I-Detailed_description, 0.4297404289...","([SEP], I-Dosage, 0.03923286125063896)",None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,"([CLS], I-Dosage, 0.03993229940533638)","(▁conformation, B-Detailed_description, 0.3937...","(▁my, I-Detailed_description, 0.6311725378036499)","(rist, I-Detailed_description, 0.6093786954879...","(oyl, I-Detailed_description, 0.7027463316917419)","(▁pocket, I-Detailed_description, 0.8434095382...","([SEP], I-Dosage, 0.040674880146980286)",None,None,None,None,None,None,None,None,None,None,None,None,None
156,"([CLS], I-Dosage, 0.03993229940533638)","(▁conformation, B-Detailed_description, 0.3937...","(▁my, I-Detailed_description, 0.6311725378036499)","(rist, I-Detailed_description, 0.6093786954879...","(oyl, I-Detailed_description, 0.7027463316917419)","(▁pocket, I-Detailed_description, 0.8434095382...","([SEP], I-Dosage, 0.040674880146980286)",None,None,None,None,None,None,None,None,None,None,None,None,None
157,"([CLS], I-Dosage, 0.03993229940533638)","(▁conformation, B-Detailed_description, 0.3937...","(▁my, I-Detailed_description, 0.6311725378036499)","(rist, I-Detailed_description, 0.6093786954879...","(oyl, I-Detailed_description, 0.7027463316917419)","(▁pocket, I-Detailed_description, 0.8434095382...","([SEP], I-Dosage, 0.040674880146980286)",None,None,None,None,None,None,None,None,None,None,N

In [3]:
# sample_text = "The patient was diagnosed with glioblastoma multiforme after the MRI scan."
sample_text = 'asciminib is an allosteric inhibitor that binds a myristoyl site of the BCR @-@ ABL1 protein , locking BCR @-@ ABL1 into an inactive conformation through a mechanism distinct from those for all other ABL kinase inhibitors .'

# Extract entities with the first model
entities_model1 = extract_entities(sample_text, model1, tokenizer1)
print("Entities from venkatd/BioMed_NER:", entities_model1)

# Extract entities with the second model
entities_model2 = extract_entities(sample_text, model2, tokenizer2)
print("Entities from d4data/biomedical-ner-all:", entities_model2)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Entities from venkatd/BioMed_NER: [('[CLS]', 'I-Dosage', 0.0401923805475235), ('ci', 'B-Medication', 0.3933495879173279), ('mini', 'B-Medication', 0.4100815951824188), ('b', 'B-Medication', 0.29926082491874695), ('[SEP]', 'I-Dosage', 0.04075567051768303)]
Entities from d4data/biomedical-ner-all: [('as', 'B-Medication', 0.9863074421882629), ('##ci', 'B-Medication', 0.9775426387786865), ('##mini', 'B-Medication', 0.8706826567649841), ('##b', 'I-Medication', 0.6157124042510986), ('all', 'B-Detailed_description', 0.9982280135154724), ('##ost', 'I-Detailed_description', 0.9995230436325073), ('##eric', 'I-Detailed_description', 0.9991564750671387), ('inhibitor', 'I-Detailed_description', 0.9990715980529785), ('bc', 'B-Detailed_description', 0.7472715973854065), ('inactive', 'I-Detailed_description', 0.7574211359024048), ('conform', 'I-Detailed_description', 0.9993818998336792), ('##ation', 'I-Detailed_description', 0.9993265867233276)]


In [ ]:
sample_text = "The patient was diagnosed with glioblastoma multiforme after the MRI scan."

# Extract entities with the first model
entities_model1 = extract_entities(sample_text, model1, tokenizer1)
print("Entities from venkatd/BioMed_NER:", entities_model1)

# Extract entities with the second model
entities_model2 = extract_entities(sample_text, model2, tokenizer2)
print("Entities from d4data/biomedical-ner-all:", entities_model2)


Entities from venkatd/BioMed_NER: [('[CLS]', 'I-Dosage', 0.039715733379125595), ('▁glioblastoma', 'B-Disease_disorder', 0.893625020980835), ('▁multi', 'I-Disease_disorder', 0.7914446592330933), ('forme', 'I-Disease_disorder', 0.6416245698928833), ('▁MRI', 'B-Diagnostic_procedure', 0.47890275716781616), ('[SEP]', 'I-Dosage', 0.039622243493795395)]
Entities from d4data/biomedical-ner-all: [('g', 'B-Disease_disorder', 0.9998356103897095), ('##lio', 'I-Disease_disorder', 0.9896998405456543), ('##bla', 'I-Disease_disorder', 0.998340368270874), ('##sto', 'I-Disease_disorder', 0.926582396030426), ('##ma', 'I-Disease_disorder', 0.9996727705001831), ('multi', 'B-Detailed_description', 0.9925978183746338), ('##form', 'I-Detailed_description', 0.9980612397193909), ('##e', 'I-Disease_disorder', 0.9973832964897156), ('mri', 'B-Diagnostic_procedure', 0.9999439716339111), ('[SEP]', 'B-Administration', 0.08152436465024948)]


In [18]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

# Load the BioBERT model and tokenizer
model_name = "dmis-lab/biobert-base-cased-v1.2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
def extract_entities(text, model, tokenizer):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    
    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs).logits
    
    # Get the predicted token labels
    predictions = torch.argmax(outputs, dim=-1)

    # Convert token IDs back to tokens
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

    # Get model labels (you can adjust based on the model’s configuration)
    labels = model.config.id2label
    
    # Extract named entities
    entities = []
    for token, prediction in zip(tokens, predictions[0]):
        if prediction != 0:  # Assuming '0' is the 'O' tag for non-entities
            entities.append((token, labels[prediction.item()]))
    
    return entities


In [20]:
sample_text = "The patient was diagnosed with glioblastoma multiforme after the MRI scan."

# Extract entities using BioBERT
entities = extract_entities(sample_text, model, tokenizer)
print("Entities extracted by BioBERT:", entities)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Entities extracted by BioBERT: [('the', 'LABEL_1'), ('patient', 'LABEL_1'), ('was', 'LABEL_1'), ('diagnosed', 'LABEL_1'), ('##blast', 'LABEL_1'), ('.', 'LABEL_1'), ('[SEP]', 'LABEL_1')]


In [21]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

# Load the Kushtrim/bert-base-cased-biomedical-ner model and tokenizer
model_name = "Kushtrim/bert-base-cased-biomedical-ner"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

def extract_entities(text, model, tokenizer):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    
    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs).logits
    
    # Get the predicted token labels
    predictions = torch.argmax(outputs, dim=-1)

    # Convert token IDs back to tokens
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

    # Get model labels (you can adjust based on the model’s configuration)
    labels = model.config.id2label
    
    # Extract named entities
    entities = []
    for token, prediction in zip(tokens, predictions[0]):
        if prediction != 0:  # Assuming '0' is the 'O' tag for non-entities
            entities.append((token, labels[prediction.item()]))
    
    return entities

# Test the model on a sample biomedical text
sample_text = "The patient was diagnosed with glioblastoma multiforme after the MRI scan."

# Extract entities using Kushtrim/bert-base-cased-biomedical-ner
entities = extract_entities(sample_text, model, tokenizer)
print("Entities extracted by Kushtrim/bert-base-cased-biomedical-ner:", entities)


Entities extracted by Kushtrim/bert-base-cased-biomedical-ner: [('g', 'B-DISEASE'), ('##lio', 'B-DISEASE'), ('##blast', 'B-DISEASE'), ('##oma', 'B-DISEASE'), ('MR', 'B-EXP_ASSAY'), ('##I', 'B-EXP_ASSAY')]


In [ ]:
token = "INSERT YOUR OWN TOKEN"
from huggingface_hub import login

# Log in to Hugging Face
login(token=token)
